# HTTP requests to the NASA Kennedy Space Center WWW server
Fonte oficial do dateset : http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html

Dados :

● Jul 01 to Jul 31, ASCII format, 20.7 MB gzip compressed , 205.2 MB.

● Aug 04 to Aug 31, ASCII format, 21.8 MB gzip compressed , 167.8 MB.

Sobre o dataset : Esses dois conjuntos de dados possuem todas as requisições HTTP para o servidor da NASA Kennedy
Space Center WWW na Flórida para um período específico.
Os logs estão em arquivos ASCII com uma linha por requisição com as seguintes colunas:

● Host fazendo a requisição . Um hostname quando possível, caso contrário o endereço de internet se o nome
não puder ser identificado.

● Timestamp no formato "DIA/MÊS/ANO:HH:MM:SS TIMEZONE"

● Requisição (entre aspas)

● Código do retorno HTTP

● Total de bytes retornados

Questões:
Responda as seguintes questões devem ser desenvolvidas em Spark utilizando a sua linguagem de preferência.

1. Número de hosts únicos.
2. O total de erros 404.
3. Os 5 URLs que mais causaram erro 404.
4. Quantidade de erros 404 por dia.
5. O total de bytes retornados.

In [1]:
#Verificando se o SparkContext foi iniciado - inicia automaticamente quando o 
#jupyter notebook para pyspark é aberto pela linha de comando na pasta do projeto.
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
#Verificando os dados do SparkContext (master e appName)
print(sc)

#Verificando a versão utilizada 
print(sc.version)

<SparkContext master=local[*] appName=PySparkShell>
2.4.5


In [3]:
#Importando sql
from pyspark import sql 
from pyspark.sql import *

#Iniciando a sessão
spark = SparkSession.builder.master("local").appName("RespostaLilian").getOrCreate()
sc = SparkContext.getOrCreate()

#Verificando a sessão do SparkSession
print(spark)

In [4]:
#Carregando os arquivos e unindo ambos em um único Dataframe com o nome de 'df'
data_jul = spark.read.load("C:\\Spark\\projetos\\dataset\\NASA_access_log_Jul95.gz",
                     format="text", sep=",", inferSchema="true", header="false") 

data_aug = spark.read.load("C:\\Spark\\projetos\\dataset\\NASA_access_log_Aug95.gz",
                     format="text", sep=",", inferSchema="true", header="false") 

df = data_jul.union(data_aug)

#Conferindo o número de linhas dos arquivos separados e após a união
print('linhas_arquivo_julho = ', data_jul.count())

print('linhas_arquivo_agosto = ', data_aug.count())

print('linhas_arquivo_total = ', df.count())

linhas_arquivo_julho =  1891715
linhas_arquivo_agosto =  1569898
linhas_arquivo_total =  3461613


In [5]:
#Conferindo o tipo do df
type(df)

#Conferindo o schema do df
df.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
#Verificando as 5 primeiras linhas do df
df.take(5)

[Row(value='199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'),
 Row(value='unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179')]

In [7]:
#Importando as funções do sql para utilizar regex
from pyspark.sql.functions import *

#Separando por vírgula as colunas desejadas do df - utilizando regex
df_sep = df.withColumn('value', regexp_replace('value', '(.*) - - \[(.*)\] "(.*)" (\d*) (.*)', '$1,$2,$3,$4,$5'))

df_sep.take(5)

[Row(value='199.72.81.55,01/Jul/1995:00:00:01 -0400,GET /history/apollo/ HTTP/1.0,200,6245'),
 Row(value='unicomp6.unicomp.net,01/Jul/1995:00:00:06 -0400,GET /shuttle/countdown/ HTTP/1.0,200,3985'),
 Row(value='199.120.110.21,01/Jul/1995:00:00:09 -0400,GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0,200,4085'),
 Row(value='burger.letters.com,01/Jul/1995:00:00:11 -0400,GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0'),
 Row(value='199.120.110.21,01/Jul/1995:00:00:11 -0400,GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0,200,4179')]

In [8]:
#Separando as colunas e nomeando-as
col_names = ("Host","Time","Req","Code","Total")

df_col = df_sep.withColumn("value", split("value",","))

df_name = df_col.select(*(col('value').getItem(i).alias(col_names[i]) for i in range(5)))

df_name.show(10)

+--------------------+--------------------+--------------------+----+-----+
|                Host|                Time|                 Req|Code|Total|
+--------------------+--------------------+--------------------+----+-----+
|        199.72.81.55|01/Jul/1995:00:00...|GET /history/apol...| 200| 6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00...|GET /shuttle/coun...| 200| 3985|
|      199.120.110.21|01/Jul/1995:00:00...|GET /shuttle/miss...| 200| 4085|
|  burger.letters.com|01/Jul/1995:00:00...|GET /shuttle/coun...| 304|    0|
|      199.120.110.21|01/Jul/1995:00:00...|GET /shuttle/miss...| 200| 4179|
|  burger.letters.com|01/Jul/1995:00:00...|GET /images/NASA-...| 304|    0|
|  burger.letters.com|01/Jul/1995:00:00...|GET /shuttle/coun...| 200|    0|
|     205.212.115.106|01/Jul/1995:00:00...|GET /shuttle/coun...| 200| 3985|
|         d104.aa.net|01/Jul/1995:00:00...|GET /shuttle/coun...| 200| 3985|
|      129.94.144.152|01/Jul/1995:00:00...|      GET / HTTP/1.0| 200| 7074|
+-----------

In [9]:
#Conferindo o schema e a contagem
df_name.printSchema()

df_name.count()

root
 |-- Host: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Req: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Total: string (nullable = true)



3461613

In [10]:
#Fazendo a alteração do schema
df_name = df_name.select(df_name.Host, 
                         df_name.Time,
                         df_name.Req,
                         df_name.Code,
                         df_name.Total.cast("integer")
                        )

df_name.printSchema()

root
 |-- Host: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Req: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Total: integer (nullable = true)



In [11]:
#Criando uma tabela temporária para poder trabalhar com spark sql
df_name.createOrReplaceTempView('tabela')


In [12]:
#Respondendo questão 1
spark.sql("SELECT COUNT(DISTINCT Host) AS No_hosts_unicos FROM tabela").show()

+---------------+
|No_hosts_unicos|
+---------------+
|         137979|
+---------------+



In [13]:
#Respondendo a questão 2
spark.sql("SELECT COUNT(Code) AS Total_erros_404 FROM tabela WHERE Code==404").show()

+---------------+
|Total_erros_404|
+---------------+
|          20848|
+---------------+



In [14]:
#Respondendo a questão 3
spark.sql("SELECT Host,COUNT(Host) FROM tabela WHERE Code==404 GROUP BY Host ORDER BY COUNT(Host) DESC ").show(5)

+--------------------+-----------+
|                Host|count(Host)|
+--------------------+-----------+
|hoohoo.ncsa.uiuc.edu|        251|
|piweba3y.prodigy.com|        157|
|jbiagioni.npt.nuw...|        132|
|piweba1y.prodigy.com|        114|
|www-d4.proxy.aol.com|         91|
+--------------------+-----------+
only showing top 5 rows



In [15]:
#Respondendo a questão 4 (mostrando as primeiras 20 linhas)
numero4 = df_name.select(df_name.Time.substr(0,11).alias("Date"),"Code")

numero4 = numero4.filter(numero4.Code.like('404'))

numero4 = numero4.groupBy("Date").count().show()

+-----------+-----+
|       Date|count|
+-----------+-----+
|02/Jul/1995|  291|
|21/Aug/1995|  305|
|06/Aug/1995|  371|
|16/Jul/1995|  257|
|07/Aug/1995|  537|
|11/Aug/1995|  263|
|27/Jul/1995|  336|
|07/Jul/1995|  566|
|17/Jul/1995|  403|
|15/Jul/1995|  254|
|18/Jul/1995|  465|
|26/Jul/1995|  336|
|03/Aug/1995|  304|
|18/Aug/1995|  256|
|17/Aug/1995|  270|
|14/Aug/1995|  287|
|10/Jul/1995|  398|
|04/Jul/1995|  358|
|20/Aug/1995|  311|
|20/Jul/1995|  419|
+-----------+-----+
only showing top 20 rows



In [16]:
#Respondendo a questão 5
spark.sql("SELECT SUM(Total) AS total_bytes_retornados FROM tabela").show()

+----------------------+
|total_bytes_retornados|
+----------------------+
|           65535001345|
+----------------------+

